## LightGBM Hyperparameter Tuning (Recall)

In [18]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier

import optuna

In [19]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [20]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [28]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', -1, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 40),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0)
    }

    model = LGBMClassifier(
        **params,
        objective='binary',
        verbose=-1,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.34
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [30]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 23:17:59,709] A new study created in memory with name: no-name-3fc562ac-c712-4f5e-abff-86516ccce436


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 23:18:00,320] Trial 0 finished with value: 0.601840490797546 and parameters: {'n_estimators': 185, 'max_depth': 17, 'learning_rate': 0.05904930722364741, 'num_leaves': 28, 'colsample_bytree': 0.9707517391192666, 'subsample': 0.7313448319899061}. Best is trial 0 with value: 0.601840490797546.
[I 2024-07-08 23:18:02,347] Trial 1 finished with value: 0.5478527607361963 and parameters: {'n_estimators': 374, 'max_depth': 6, 'learning_rate': 0.21542131568016862, 'num_leaves': 27, 'colsample_bytree': 0.9124639575567663, 'subsample': 0.9833348995176587}. Best is trial 0 with value: 0.601840490797546.
[I 2024-07-08 23:18:03,728] Trial 2 finished with value: 0.5576687116564416 and parameters: {'n_estimators': 456, 'max_depth': 43, 'learning_rate': 0.14465049545002429, 'num_leaves': 30, 'colsample_bytree': 0.5432017111575661, 'subsample': 0.6527576027695939}. Best is trial 0 with value: 0.601840490797546.
[I 2024-07-08 23:18:04,493] Trial 3 finished with value: 0.5901840490797545 an

In [33]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=89, state=1, values=[0.6073619631901841], datetime_start=datetime.datetime(2024, 7, 8, 23, 20, 8, 842436), datetime_complete=datetime.datetime(2024, 7, 8, 23, 20, 9, 741098), params={'n_estimators': 322, 'max_depth': 0, 'learning_rate': 0.03335525725756855, 'num_leaves': 26, 'colsample_bytree': 0.6813157390893247, 'subsample': 0.8853493442333064}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'max_depth': IntDistribution(high=50, log=False, low=-1, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'num_leaves': IntDistribution(high=40, log=False, low=20, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None)}, trial_id=89, value=None)
Best hyperparameters: {'n_estimators': 322, 'max_depth': 0, 'learning_rate': 0.0333552